# Астероиды и кометы

## Заложим основы

In [43]:
import numba as nb
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import unittest
from pathlib import Path
from PIL import Image
from IPython.display import clear_output
from IPython.core.interactiveshell import InteractiveShell
from IPython import display
from tqdm import tqdm
from dataclasses import dataclass

dt = 1e-3
G = 6,7e-11
au = 149597870700

InteractiveShell.ast_node_interactivity = "all"

### Зададим объект
В моей программе только один класс, поскольку принципиальных различий между реальными небесными телами нет

In [44]:
@dataclass
class CosmicBody:   
    existance: bool
    mass: float
    obj_radius: float
    radius_vec: np.array
    velocity_vec: np.array


### Зададим методы
Их будет всего два:
- Рассчёт изменений всех параметров системы за dt (учитывает 2 возможных сценария: смещение и столкновение)
- Определение типа траектории

In [47]:
def dtChange(syst):
    for i in range(len(syst)):
        syst[i].radius_vec += np.dot(syst[i].velocity_vec, dt)
        
    for i in range(len(syst)):
        for j in range(len(syst)) :
            if j != i:
                syst[i].velocity_vec += np.dot(G*dt/np.linalg.norm(syst[j].radius_vec - syst[i].radius_vec)**3 , syst[j].mass*(syst[j].radius_vec - syst[i].radius_vec))
    
    for i in range(len(syst)):
        for j in range(len(syst)) :
            if j != i:
                if np.linalg.norm(syst[j].radius_vec - syst[i].radius_vec) < syst[i].obj_radius | np.linalg.norm(syst[j].radius_vec - syst[i].radius_vec) < syst[j].obj_radius:
                    syst[i].mass += syst[j].mass
                    syst[i].radius_vec += np.dot(syst[j].mass/(syst[j].mass+syst[i].mass), (syst[j].radius_vec - syst[i].radius_vec))
                    syst[i].obj_radius *= (1 + (syst[i].mass / syst[j].mass)) ** (1/3)
                    syst[j].mass = 0
                    syst[j].obj_radius = 0
                    syst[j].velocity_vec = 0
                    syst[j].radius_vec = 0


Big   = CosmicBody(1, 100, 10, [0,0,0],     [0, 0, 0])
Body1 = CosmicBody(1, 0.5, 1,  [15, 1, 0],  [0, -0.5, 10])
Body2 = CosmicBody(1, 20,  30, [0, 3, 50],  [20, -5, 0])
Body3 = CosmicBody(1, 5,   2,  [10, 0, 12], [10, 0, -10])

dtChange(syst=[Big, Body1, Body2, Body3])

TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
'''Big   = CosmicBody(1, 100, 10, [0,0,0],     [0, 0, 0])
Body1 = CosmicBody(1, 0.5, 1,  [15, 1, 0],  [0, -0.5, 10])
Body2 = CosmicBody(1, 20,  30, [0, 3, 50],  [20, -5, 0])
Body3 = CosmicBody(1, 5,   2,  [10, 0, 12], [10, 0, -10])

dtChange(syst=[Big, Body1, Body2, Body3])'''

NameError: name 'dtChange' is not defined

In [ ]:
def plotter(syst):
    """
    The basic function to plot orbits of bodies.
    Can be used for 2 and 3 dimensional space.
    Show plot.

    Parameters
    ----------
    dim : int64 (2 or 3)
        Dimension of space
    Sun : object of class Body
    syst : list of objects of class Body
    """
    
    fig = plt.figure(figsize=(100, 100))
    ax = fig.add_subplot(111, projection='3d')
    ax.axes.set_xlim3d(-100, 100)
    ax.axes.set_ylim3d(-100, 100)
    ax.axes.set_zlim3d(-80, 80)

    '''u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)

    x = Sun.R * np.outer(np.cos(u), np.sin(v))
    y = Sun.R * np.outer(np.sin(u), np.sin(v))
    z = Sun.R * np.outer(np.ones(np.size(u)), np.cos(v))
    ax.plot_surface(x, y, z, rstride=4, cstride=4, color='r', linewidth=0, alpha=0.5)
    ax.plot_surface(x, y, -z, rstride=4, cstride=4, color='r', linewidth=0, alpha=0.5)
    '''
    for i in range(len(syst)):
        plt.scatter(syst[i].radius_vec[:, 0], syst[i].radius_vec[:, 1], syst[i].radius_vec[:, 2], s=1)

    plt.show()